In [10]:
from lib import vrep
import sys, time
from src import robot as rb
from src.utils import vrep2array
import math
from time import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Defining the kinematic model of the Pionner P3DX
For this project, we are going to use the configuration of the mobile robot being characterized by the position (x,y) and the orientation in a Cartesian coordinate.

Using the follow parameters:
1. $V_R$: linear velocity of the right wheel.
2. $V_L$: linear velocity of the left wheel.
3. $W$: angular velocity of the mobile robot.
4. $X$: abscissa of the robot.
5. $Y$: intercept of the robot.
6. $X,Y$ : the actual position coordinates.
7. $\theta$: orientation of the robot.
8. $L$: the distance between the driving wheels.

The kinematic model is given by these equations [1](https://www.hindawi.com/journals/cin/2016/9548482/abs/):
<br>
\begin{align}
\frac{dX}{dt} & = \frac{V_L + V_R}{2} \cdot cos(\theta) \\
\frac{dY}{dt} & = \frac{V_L + V_R}{2} \cdot sen(\theta) \\
\frac{d \theta}{dt} & = \frac{V_L - V_R}{2} \\
\end{align}
<br>
Where ($X$,$Y$ and $\theta$) are the robot actual position and orientation angle in world reference frame. In simulation, we use the discrete form to build a model of the robot. The discrete form of the kinematic model is given by the following equations:<br>
<br>
\begin{align}
X_{k+1} & = X_k + T \cdot \frac{V_{lk} + V_{rk}}{2} \cdot cos(\theta_k) \\
Y_{k+1} & = Y_k + T \cdot \frac{V_{lk} + V_{rk}}{2} \cdot sen(\theta_k) \\
\theta_{k+1} & = \theta_k + T \cdot \frac{V_{lk} + V_{rk}}{L} \\
\end{align}
<br>

where $X_{k+1}$ and $Y_{k+1}$ represent the position of the center axis of the mobile robot and $T$ is the sampling time.

In [3]:
class Pose:
    def __init__(self, x=None, y=None, orientation=None):
        self.x = x
        self.y = y
        self.orientation = orientation

In [4]:
class Odometry():
    """
    A class used to implement methods that allow a robot to calculate his own odometry.

    ...

    Attributes
    ----------
    robot : obj
        The robot object
    lastPose : obj Pose
        Store the robot's pose during his movement
    lastTimestamp : time
        Store the last timestamp
    left_vel : double
        Store the velocity of the left robot wheel
    right_vel : double
        Store the velocity of the right robot wheel
    delta_time : double
        Store how much time has passed
    delta_theta : double
        Store how the orientation change
    delta_space : double
        Store how the (x,y) change

    Methods
    -------
    ground_truth_updater()
        Function to update the ground truth, the real pose of the robot at the simulator
    odometry_pose_updater()
        Function to estimate the pose of the robot based on the kinematic model 
    """
    def __init__(self, robot):
        self.robot = robot
        self.lastPose = None
        self.lastTimestamp = time()
        self.left_vel = 0
        self.right_vel = 0
        self.delta_time = 0
        self.delta_theta = 0
        self.delta_space = 0
        
    def ground_truth_updater(self):
        """
            Function to update the ground truth, the real pose of the robot at the simulator
        """
        # get the (x,y,z) position of the robot at the simulator
        pose = self.robot.get_current_position()
        # get the orientation of the robot (euler angles)
        orientation = self.robot.get_current_orientation()
        
        # return an pose object (x,y,theta)
        return Pose(x=pose[0], y=pose[1], orientation=orientation[2])
    
    def odometry_pose_updater(self):
        """
            Function to estimate the pose of the robot based on the knematic model
        """
        if self.lastPose is None:
            self.lastPose = self.ground_truth_updater()
            return self.lastPose
            
        # get the actual timestamp
        time_now = time()
        # get the robot linear velocity for the left and right wheel
        left_vel, right_vel = self.robot.get_linear_velocity()
        # calculate the difference between the acutal and last timestamp
        delta_time = time_now - self.lastTimestamp
        # calculate the angle deslocation - based on the kinematic model
        delta_theta = (right_vel + left_vel) * (delta_time / self.robot.ROBOT_WIDTH)
        # calculate the distance deslocation - based on the kinematic model
        delta_space = (right_vel + left_vel) * (delta_time / 2)
        
        # auxiliary function to sum angles
        add_deltha = lambda start, delta: (((start+delta)%(2*math.pi))-(2*math.pi)) if (((start+delta)%(2*math.pi))>math.pi) else ((start+delta)%(2*math.pi))

        # calculate the new X pose
        x = self.lastPose.x + (delta_space * math.cos(self.lastPose.orientation))
        # calculate the new Y pose
        y = self.lastPose.y + (delta_space * math.sin(self.lastPose.orientation))
        # calculate the new Orientation pose
        theta = add_deltha(self.lastPose.orientation, delta_theta)
        
        # uptade the state of the class
        self.lastPose = Pose(x, y, theta)
        self.lastTimestamp = time_now
        self.left_vel = left_vel
        self.right_vel = right_vel
        self.delta_time = delta_time
        self.delta_theta = delta_theta
        self.delta_space = delta_space
        
        return self.lastPose

In [5]:
class Walker():
    """
    A class used to implement methods that allow a robot to walk, based on different behaviors.

    ...

    Attributes
    ----------
    robot : obj
        the robot object

    Methods
    -------
    find_wall()
        Function that aims to find a wall in the front of the robot.
    turn_left()
        Function that aims to make the robot turn left.
    follow_wall()
        Function that aims to make the robot follow the wall that stays at his left side.
    """
    def __init__(self, robot):
        """
            Instanciate the object
        """
        self.robot = robot
    
    def find_wall(self):
        """
            Function that aims to find a wall in the front of the robot.
        """
        #  read the sonars
        sensors = self.robot.read_ultrassonic_sensors()
        
        # loop to make the robot walk until he finds a wall
        if sensors[3] > 0.45 and sensors[4] > 0.45:
            # set the initial velocity
            self.robot.set_left_velocity(3)
            self.robot.set_right_velocity(3)
            
            return False
        else:        
            return True
    
    def turn_left(self):
        """
            Function that aims to make the robot turn left.
        """
        # read the sonar sensors
        sensors = self.robot.read_ultrassonic_sensors()

        if (sensors[0] > 0.5 and sensors[15] > 0.5) or (sensors[0] - sensors[15] > 0.1) or (sensors[15] - sensors[0] > 0.1):
            # start the motors velocity
            self.robot.set_left_velocity(1.5)
            self.robot.set_right_velocity(0)
            
            return False
        else:
            return True
        

    def follow_wall(self):
        """
            Function that aims to make the robot follow the wall that stays at his left side.
        """
        sensors = self.robot.read_ultrassonic_sensors()

        if sensors[3] < 0.6 or sensors[4] < 0.6 or sensors[5] < 0.6 or sensors[6] < 0.6:
            self.robot.set_right_velocity(0)
            self.robot.set_left_velocity(1.5)
        elif sensors[0] < 0.37 or sensors[1] < 0.37 or sensors[2] < 0.37:
            self.robot.set_right_velocity(0)
            self.robot.set_left_velocity(1.5)
        elif sensors[0] > 0.55 or sensors[1] > 0.55:
            self.robot.set_right_velocity(1.5)
            self.robot.set_left_velocity(0)
        else:
            self.robot.set_right_velocity(1.5)
            self.robot.set_left_velocity(1.5)

        return

DAQUI PRA BAIXO É TESTE

In [6]:
class Frame():
    def __init__(self, pose, image, timestamp=None):
        self. pose = pose
        self.image = image
        self.timestamp = time()

In [7]:
class FeatureExtractor():
    def __init__(self, robot):
        self.robot = robot
        self.first_image = None
        self.second_image = None
        
        
    def step(self):
        self.update()
        if not (self.first_image is None and self.second_image is None):
            print("DO SOMETHING!")
        
        
    def update(self):
        # get distance between robot and a wall
        self.wall_distance = min(self.robot.read_ultrassonic_sensors()[3:5])
        # get the (x,y,z) position of the robot at the simulator
        pose = self.robot.get_current_position()
        # get the orientation of the robot (euler angles)
        orientation = self.robot.get_current_orientation()
        
        if self.first_image is None:
            # set the first image if it's not setted yet
            # read camera
            image = self.robot.read_vision_sensor()
            image = vrep2array(image[1],image[0])
            # form pose
            pose = Pose(x=pose[0], y=pose[1], orientation=orientation[2])
            # attribute image
            self.first_image = Frame(pose=pose, image=image)
        elif self.second_image is None:
            # set the second image if it's not setted yet
            # get image
            image = self.robot.read_vision_sensor()
            image = vrep2array(image[1],image[0])
            # get pose
            pose = Pose(x=pose[0], y=pose[1], orientation=orientation[2])
            # set image
            self.second_image = Frame(pose=pose, image=image)
        else:
            # update first and second image if they both exist
            self.first_image = self.second_image
            image = self.robot.read_vision_sensor()
            image = vrep2array(image[1],image[0])
            pose = Pose(x=pose[0], y=pose[1], orientation=orientation[2])
            self.second_image = Frame(pose=pose, image=image)

In [8]:
ground_truth = []
odometry = []
images = []

In [9]:
def state_machine(behavior="follow_wall"):
    # first we create the robot and the walker object
    robot = rb.Robot()
    walker_behavior = Walker(robot)
    
    # create the FeatureExtractor robot
    feature_extractor = FeatureExtractor(robot)
    
    # instantiate the odometry calculator
    odometry_calculator = Odometry(robot=robot)
    # add the start position to both lists
    temp_start_pos = odometry_calculator.ground_truth_updater()
    ground_truth.append((temp_start_pos.x, temp_start_pos.y))
    odometry.append((temp_start_pos.x, temp_start_pos.y))

    
    if behavior == "follow_wall":

        # first we find a wall
        while not walker_behavior.find_wall():
            # read image
            feature_extractor.update()
            
            # calculate the estimate new position after find a wall
            temp_ground = odometry_calculator.ground_truth_updater()
            ground_truth.append((temp_ground.x, temp_ground.y))
            temp_odom = odometry_calculator.odometry_pose_updater()
            odometry.append((temp_odom.x, temp_odom.y))
            
        robot.stop()
    
        # then we align to the wall
        while not walker_behavior.turn_left():
            # read image
            feature_extractor.update()
            # calculate the estimate new position after find a wall
            temp_ground = odometry_calculator.ground_truth_updater()
            ground_truth.append((temp_ground.x, temp_ground.y))
            temp_odom = odometry_calculator.odometry_pose_updater()
            odometry.append((temp_odom.x, temp_odom.y))        
        
        robot.stop()

        # now we follow the wall
        while True:
            # read image
            feature_extractor.update()

            # calculate the estimate new position after find a wall
            temp_ground = odometry_calculator.ground_truth_updater()
            ground_truth.append((temp_ground.x, temp_ground.y))
            temp_odom = odometry_calculator.odometry_pose_updater()
            odometry.append((temp_odom.x, temp_odom.y))        

            # call the function that keeps following the left wall
            walker_behavior.follow_wall()
    else:
        raise Exception("Not implemented!")
        

In [ ]:
state_machine()

Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1 connected.
 Pioneer_p3dx_ultrasonicSensor2 connected.
 Pioneer_p3dx_ultrasonicSensor3 connected.
 Pioneer_p3dx_ultrasonicSensor4 connected.
 Pioneer_p3dx_ultrasonicSensor5 connected.
 Pioneer_p3dx_ultrasonicSensor6 connected.
 Pioneer_p3dx_ultrasonicSensor7 connected.
 Pioneer_p3dx_ultrasonicSensor8 connected.
 Pioneer_p3dx_ultrasonicSensor9 connected.
 Pioneer_p3dx_ultrasonicSensor10 connected.
 Pioneer_p3dx_ultrasonicSensor11 connected.
 Pioneer_p3dx_ultrasonicSensor12 connected.
 Pioneer_p3dx_ultrasonicSensor13 connected.
 Pioneer_p3dx_ultrasonicSensor14 connected.
 Pioneer_p3dx_ultrasonicSensor15 connected.
 Pioneer_p3dx_ultrasonicSensor16 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
> <ipython-input-19-041c553e7006>(30)update()
     28             pose = Pose(x=pose[0], y=pose[1], orientation=orientation[2])
     29             # attribute image